In [1]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results = Path("~/ray_results").expanduser()
assert ray_results.is_dir()
last_experiment = max(ray_results.glob('_aux_train_daostack_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

def _aux_train_daostack(*args, **kwargs):
    raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), _aux_train_daostack)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/daviddavo/ray_results/_aux_train_daostack_2023-08-04_18-10-33...


2023-08-05 14:44:10,796	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.


Result(
  metrics={'loss': 0.36464700296949626, 'rprec train': 0.4460618197917938, 'rprec test': 0.24987199902534485, 'p@5 train': 0.576923131942749, 'p@5 test': 0.5173077583312988, 'r@5 train': 0.11515726894140244, 'r@5 test': 0.11842279136180878, 'should_checkpoint': True, 'done': True, 'trial_id': '7d17313d', 'experiment_tag': '1420_trial_index=4,batch_size=9,conv_layers=4,embedding_dim=255,l2=0.0000,learning_rate=0.9984,max_epochs=50'},
  path='/home/daviddavo/ray_results/_aux_train_daostack_2023-08-04_18-10-33/_aux_train_daostack_7d17313d_1420_trial_index=4,batch_size=9,conv_layers=4,embedding_dim=255,l2=0.0000,learning_rate=0.9984,max_ep_2023-08-04_18-56-00',
  checkpoint=Checkpoint(local_path=/home/daviddavo/ray_results/_aux_train_daostack_2023-08-04_18-10-33/_aux_train_daostack_7d17313d_1420_trial_index=4,batch_size=9,conv_layers=4,embedding_dim=255,l2=0.0000,learning_rate=0.9984,max_ep_2023-08-04_18-56-00/checkpoint_000049)
)

In [3]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('p@5 test', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf.sort_values('loss').head(5)

Index(['loss', 'rprec train', 'rprec test', 'p@5 train', 'p@5 test',
       'r@5 train', 'r@5 test', 'time_this_iter_s', 'should_checkpoint',
       'done', 'training_iteration', 'trial_id', 'time_total_s',
       'config/__trial_index__', 'config/batch_size', 'config/conv_layers',
       'config/embedding_dim', 'config/l2', 'config/learning_rate',
       'config/max_epochs'],
      dtype='object')


,loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,should_checkpoint,done,training_iteration,trial_id,time_total_s,config/__trial_index__,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,config/max_epochs
1084,0.000002,0.341698,0.187381,0.428846,0.359615,0.063187,0.057253,0.242601,True,False,48,72ce5191,11.779700,1,512,2,225,1.008631e-09,0.09383,50
474,0.000003,0.333034,0.188084,0.467308,0.328846,0.070219,0.055709,0.244043,True,False,40,3012a977,9.830602,3,512,2,225,1.008631e-09,0.09383,50
108,0.000003,0.322663,0.136328,0.451923,0.226923,0.062413,0.025628,3.742894,True,False,40,093824ed,153.019838,1,32,4,395,2.291114e-09,0.02109,50
1688,0.000005,0.341493,0.178222,0.463462,0.346154,0.068529,0.052727,0.272583,True,False,22,ad73e657,6.189987,4,512,2,225,1.316051e-09,0.09943,50
1884,0.000005,0.339063,0.172414,0.480769,0.294231,0.079739,0.049389,0.420175,True,False,49,c10393ca,17.130680,1,512,5,320,1.322222e-09,0.20245,50


In [5]:
# Group by config (except trial index) and do the mean
non_config_columns = ['config/__trial_index__', 'config/max_epochs']
cfg_columns = [c for c in rdf.columns if c.startswith('config/') and c not in non_config_columns]
non_results_cols = ['should_checkpoint', 'done', 'config/__trial_index__'] + non_config_columns
rdfgb = rdf.drop(columns=non_results_cols).groupby(cfg_columns).mean(numeric_only=True).reset_index()
print(len(rdfgb), "configurations were tested")
rdfgb.sort_values('p@5 test', ascending=False).tail(30)

500 configurations were tested


,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,training_iteration,time_total_s
48,64,4,245,9.914827e-07,0.00964,0.241608,0.242316,0.098725,0.341538,0.165769,0.040000,0.015395,2.053089,16.6,34.066409
215,256,5,405,1.584187e-07,0.00185,0.150563,0.224705,0.093210,0.328077,0.165000,0.036903,0.013761,0.627006,25.6,16.211258
471,512,5,445,1.732332e-09,0.00156,0.307133,0.216381,0.096421,0.321154,0.165000,0.036596,0.017662,0.396081,27.2,9.514649
367,512,5,100,3.840663e-09,0.00847,0.194887,0.217179,0.092560,0.317692,0.164615,0.036657,0.015135,0.265311,26.6,6.984504
388,512,5,225,2.105721e-08,0.00231,0.213107,0.216814,0.096675,0.318462,0.164615,0.035424,0.015232,0.309438,31.2,9.989760
359,512,5,25,2.496945e-08,0.12938,0.256727,0.218319,0.099074,0.317308,0.164231,0.039383,0.015402,0.363311,11.2,3.916391
254,512,3,105,9.521968e-02,0.80491,2.358417,0.251073,0.085161,0.354615,0.161923,0.042400,0.014133,0.265382,29.0,7.819533
79,128,4,135,1.846316e-04,0.00185,0.243835,0.220650,0.083026,0.326923,0.160769,0.037476,0.012908,0.714866,23.8,16.914875
53,64,5,225,2.069621e-02,0.13739,1.253451,0.239358,0.088685,0.351538,0.159615,0.041451,0.016235,2.469692,11.8,28.320639
186,256,5,215,1.265913e-07,0.00361,0.150869,0.225793,0.094648,0.327308,0.159615,0.037542,0.013152,0.577506,24.2,13.803180
